In [1]:
import pandas as pd
import numpy as np #для матричных вычислений

from sklearn.model_selection import train_test_split #сплитование выборки
from sklearn import linear_model #линейные модели
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn import ensemble #ансамбли

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials

In [2]:
data=pd.read_csv('D:\Irina\Data Science\ML\Оптимизация гиперпараметров модели\data\_train_sem09 (1).csv')
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


Создаем матрицу наблюдений $X$ и вектор ответов $y$

In [3]:
X = data.drop(['Activity'], axis=1)
y = data['Activity']

Разделяем выборку на тренировочную и тестовую в соотношении 80/20. Для сохранения соотношений целевого признака используем параметр stratify (стратифицированное разбиение). 

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 42, test_size = 0.2)
print('X_train:{}'.format(X_train.shape))
print('y_train:{}'.format(y_train.shape))
print('X_test:{}'.format(X_test.shape))
print('y_test:{}'.format(y_test.shape))

X_train:(3000, 1776)
y_train:(3000,)
X_test:(751, 1776)
y_test:(751,)


Для подбора гиперпараметров разделим тестовую выборку на валидационную и тестовую.

In [5]:
X_valid, X_test_2, y_valid, y_test_2 = train_test_split(X_test, y_test, stratify=y_test, random_state = 42, test_size = 0.5)
print('X_valid:{}'.format(X_valid.shape))
print('X_test_2:{}'.format(X_test_2.shape))

X_valid:(375, 1776)
X_test_2:(376, 1776)


Обучим модель на всех данных при помощи логистической регрессии

In [6]:
#Создаем объект класса логистическая регрессия
log_reg = linear_model.LogisticRegression(max_iter=50)

#Обучаем модель
log_reg.fit(X_train, y_train)
y_train_pred=log_reg.predict(X_train)
y_test_pred = log_reg.predict(X_test)
#Выводим значения метрики
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тренировочном наборе: 0.88
f1_score на тестовом наборе: 0.78


c:\Users\oikuz\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Проделаем аналогичное для RandomForestClassifier().

In [7]:
#Создаем объект класса случайный лес
rf = ensemble.RandomForestClassifier(random_state=42)

#Обучаем модель
rf.fit(X_train, y_train)
#Выводим значения метрики 
y_train_pred = rf.predict(X_train)
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = rf.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тренировочном наборе: 1.00
f1_score на тестовом наборе: 0.80


На полном наборе данных метрика выше для RandomForestClassifier()

Подберем гиперпараметры методом GridSeachCV для линейной регрессии

In [8]:
param_grid = [
              {'penalty': ['l2', 'none'] , # тип регуляризации
              'solver': ['lbfgs', 'sag'], # алгоритм оптимизации
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}, # уровень силы регурялизации
              
              {'penalty': ['l1', 'l2'] ,
              'solver': ['liblinear', 'saga'],
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}
]
grid_search_1 = GridSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=50), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
%time grid_search_1.fit(X_valid, y_valid) 

y_test_2_pred = grid_search_1.predict(X_test_2)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test_2, y_test_2_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_1.best_params_))

CPU times: total: 1.3 s
Wall time: 14.6 s
f1_score на тестовом наборе: 0.74
Наилучшие значения гиперпараметров: {'C': 0.3, 'penalty': 'l1', 'solver': 'saga'}


c:\Users\oikuz\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Обучение на параметрах, подобранных методом GridSearchCV ухудшило метрику f1

Подберем гиперпараметры методом GridSeachCV для случайного леса.

In [9]:
param_grid = {'n_estimators': list(range(50, 200, 10)),
              'min_samples_leaf': [3, 5, 7],
              'max_depth': list(np.linspace(10, 50, 5, dtype=int))
              }
            
grid_search_forest = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
%time grid_search_forest.fit(X_valid, y_valid) 
y_test_2_pred = grid_search_forest.predict(X_test_2)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test_2, y_test_2_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_forest.best_params_))

CPU times: total: 5.75 s
Wall time: 1min 8s
f1_score на тестовом наборе: 0.77
Наилучшие значения гиперпараметров: {'max_depth': 20, 'min_samples_leaf': 3, 'n_estimators': 110}


Для случайного леса значение метрики так же ухудшилось

Подберем гиперпараметры методом RandomizedSearchCV для линейной регрессии

In [10]:
param_distributions = {'penalty': ['l1', 'l2'] ,
              'solver': ['liblinear', 'saga'],
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]},
            
random_search = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=50), 
    param_distributions=param_distributions, 
    cv=5, 
    n_iter = 10, 
    n_jobs = -1
)  
%time random_search.fit(X_valid, y_valid) 
y_test_2_pred = random_search.predict(X_test_2)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test_2, y_test_2_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search.best_params_))

CPU times: total: 609 ms
Wall time: 3.24 s
f1_score на тестовом наборе: 0.74
Наилучшие значения гиперпараметров: {'solver': 'saga', 'penalty': 'l1', 'C': 0.3}


c:\Users\oikuz\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Подбор параметров методом RandomizedSearchCV для линейной регрессии на данный момент показал самые низкие результаты.

Подберем гиперпараметры методом RandomizedSearchCV для случайного леса

In [11]:
param_distributions = {'n_estimators': list(range(50, 200, 10)),
                       'min_samples_leaf': [3, 5, 7],
                       'max_depth': list(np.linspace(10, 50, 5, dtype=int))
                       }
            
random_search_forest = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_distributions=param_distributions, 
    cv=5,
    n_iter = 10, 
    n_jobs = -1
)  
%time random_search_forest.fit(X_valid, y_valid) 
y_test_2_pred = random_search_forest.predict(X_test_2)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test_2, y_test_2_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_forest.best_params_))

CPU times: total: 469 ms
Wall time: 3.14 s
f1_score на тестовом наборе: 0.76
Наилучшие значения гиперпараметров: {'n_estimators': 150, 'min_samples_leaf': 5, 'max_depth': 30}


Для случайного леса получили метрику чуть лучше, чем при методе GridSeachCV, но значительно быстрее.

Подберем гиперпараметры методом Hyperopt для линейной регрессии

In [15]:
random_state = 42
def hyperopt_lr(params, cv=5, X=X_valid, y=y_valid, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'penalty': str(params['penalty']), 
              'solver': str(params['solver']), 
             'C': float(params['C'])
              }
  
    # используем эту комбинацию для построения модели
    model = linear_model.LogisticRegression(**params, random_state=random_state)

    # обучаем модель
    model.fit(X, y)
    score = metrics.f1_score(y,model.predict(X))
    #score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [26]:
%%time

# зададим пространство поиска гиперпараметров
space_lr = hp.choice(
    'logreg_params',
    [
        {
            'penalty': hp.choice('penalty_1', ['l2', 'none']),
            'solver': hp.choice('solver_1', ['lbfgs', 'sag']),
            'C': hp.choice('C_1', list(np.linspace(0.01, 1, 10, dtype=float)))
        },
        {
            'penalty': hp.choice('penalty_2', ['l1', 'l2']),
            'solver': hp.choice('solver_2', ['liblinear', 'saga']),
            'C': hp.choice('C_2', list(np.linspace(0.01, 1, 10, dtype=float)))
        }
    ]
)

trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_lr, # наша функция 
          space=space_lr, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=30, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

model = linear_model.LogisticRegression(
    random_state = random_state, 
    penalty = str(best['penalty']),
    solver = str(best['solver']),
    C = float(best['C'])
)
model.fit(X_valid, y_valid)
y_valid_pred = model.predict(X_train_scaled)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_valid, y_valid_pred)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))


  0%|          | 0/30 [00:00<?, ?trial/s, best loss=?]

c:\Users\oikuz\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(



 10%|█         | 3/30 [00:00<00:06,  4.19trial/s, best loss: -0.9950980392156863]

c:\Users\oikuz\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(

c:\Users\oikuz\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(



 17%|█▋        | 5/30 [00:01<00:07,  3.53trial/s, best loss: -0.9950980392156863]

...

 93%|█████████▎| 28/30 [00:07<00:00,  9.79trial/s, best loss: -1.0]

c:\Users\oikuz\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(

c:\Users\oikuz\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(

c:\Users\oikuz\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(



100%|██████████| 30/30 [00:07<00:00,  4.09trial/s, best loss: -1.0]
Наилучшие значения гиперпараметров {'C_1': 8, 'logreg_params': 0, 'penalty_1': 1, 'solver_1': 0}


c:\Users\oikuz\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(



KeyError: 'penalty'